Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [18]:
from itertools import product, combinations
import numpy as np
import networkx as nx
from icecream import ic
import heapq 
from typing import Tuple, List, Optional, Union

In [21]:
def create_problem(
    size: int,
    *,
    density: float = 1.0,
    negative_values: bool = False,
    noise_level: float = 0.0,
    seed: int = 42,
) -> np.ndarray:
    """Problem generator for Lab3"""
    rng = np.random.default_rng(seed)
    map = rng.random(size=(size, 2))
    problem = rng.random((size, size))
    if negative_values:
        problem = problem * 2 - 1
    problem *= noise_level
    for a, b in product(range(size), repeat=2):
        if rng.random() < density:
            problem[a, b] += np.sqrt(
                np.square(map[a, 0] - map[b, 0]) + np.square(map[a, 1] - map[b, 1])
            )
        else:
            problem[a, b] = np.inf
    np.fill_diagonal(problem, 0)
    return (problem * 1_000).round()

In [22]:
#size: 10,20,50,100,200,500,1000
#density: 0.2,  0.5, 0.8, 1.0
#noise_level: 0, 0.1, 0.5, 0.8
#negative_values: True, False
problem = create_problem(10, density=0.15, noise_level=10, negative_values=False)

In [23]:
masked = np.ma.masked_array(problem, mask=np.isinf(problem))
G = nx.from_numpy_array(masked, create_using=nx.DiGraph)

In [ ]:
def solver(problem: np.ndarray, source: int, target: int) -> Tuple[Optional[List[int]], Union[float, int]]:
    num_nodes = len(problem)

    dist = {node: np.inf for node in range(num_nodes)}
    parent = {node: None for node in range(num_nodes)}
    relax_count = {node: 0 for node in range(num_nodes)}

    dist[source] = 0
    
    pq = [(0, source)]  # (g(source), source)
    
    while pq:
        # Estrai il nodo con g_score più basso
        current_g, current_node = heapq.heappop(pq)
        
        # Ottimizzazione: se abbiamo trovato un percorso più breve per questo nodo
        # *dopo* averlo aggiunto alla coda, questa voce è "vecchia". Saltiamola.
        if current_g > dist[current_node]:
            continue
            
        # 4. Rilassamento degli archi (come Bellman-Ford)
        for neighbor in range(num_nodes):
            # Se esiste un arco (costo non infinito)
            if cost != np.inf:
                new_dist_g = current_g + cost
                
                # Trovato un percorso più breve?
                if new_dist_g < dist[neighbor]:
                    parent[neighbor] = current_node
                    
                    # 5. Controllo Ciclo Negativo (come SPFA)
                    relax_count[neighbor] += 1
                    if relax_count[neighbor] > num_nodes:
                        # Rilevato un ciclo negativo!
                        return None, -np.inf
                    
                    # 6. Ri-aggiungi alla coda (con g_score come priorità)
                    #    Aggiungiamo il vicino con il nuovo costo g migliore.
                    heapq.heappush(pq, (new_dist_g, neighbor))
    if dist[target] == np.inf:
        return None, np.inf  
    curr = target
    while curr is not None:
        if curr == source:
            break
        curr = parent[curr]
        
        if curr is None and path[-1] != source:
            return None, np.inf 

    return path[::-1], dist[target]

In [31]:
for s, d in combinations(range(problem.shape[0]), 2):
    try:
        #path = nx.shortest_path(G, s, d, weight='weight')
        path = nx.bellman_ford_path(G, s, d, weight='weight') 
        cost = cost = nx.path_weight(G, path, weight='weight')
        path1, cost1 = solver(problem, s, d)
    except nx.NetworkXNoPath:
        # Nodes are not connected
        path = None
        cost = np.inf
        path1 = None
        cost1 = np.inf
    except nx.NetworkXUnbounded:
        # Negative cycle detected
        path = None
        cost = -np.inf
        path1 = None
        cost1 = -np.inf
    ic(s, d, cost,  cost1)
None

ic| s: 0, d: 1, cost: inf, cost1: inf
ic| s: 0, d: 2, cost: 6118.0, cost1: -inf
ic| s: 0, d: 3, cost: 12420.0, cost1: -inf
ic| s: 0, d: 4, cost: 8430.0, cost1: -inf
ic| s: 0, d: 5, cost: inf, cost1: inf
ic| s: 0, d: 6, cost: 7235.0, cost1: -inf
ic| s: 0, d: 7, cost: 831.0, cost1: -inf
ic| s: 0, d: 8, cost: 1977.0, cost1: -inf
ic| s: 0, d: 9, cost: 8326.0, cost1: -inf
ic| s: 1, d: 2, cost: 19810.0, cost1: -inf
ic| s: 1, d: 3, cost: inf, cost1: inf
ic| s: 1, d: 4, cost: inf, cost1: inf
ic| s: 1, d: 5, cost: 2434.0, cost1: -inf
ic| s: 1, d: 6, cost: 14621.0, cost1: -inf
ic| s: 1, d: 7, cost: 14523.0, cost1: -inf
ic| s: 1, d: 8, cost: inf, cost1: inf
ic| s: 1, d: 9, cost: 6771.0, cost1: -inf
ic| s: 2, d: 3, cost: inf, cost1: inf
ic| s: 2, d: 4, cost: inf, cost1: inf
ic| s: 2, d: 5, cost: inf, cost1: inf
ic| s: 2, d: 6, cost: 10058.0, cost1: -inf
ic| s: 2, d: 7, cost: 9960.0, cost1: -inf
ic| s: 2, d: 8, cost: inf, cost1: inf
ic| s: 2, d: 9, cost: 2208.0, cost1: -inf
ic| s: 3, d: 4, cost: 77